##### Import libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader

/home/felix.blanco/.conda/envs/BT/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### Import own libraries

In [2]:
from dataloader.iris import IrisDataset
from model.iris_cnn import IrisCNN

##### Load dataset

In [3]:
iris_df = pd.read_csv('../data/iris.data', header = None)

# Divide the dataset into 'features' and 'labels' arrays
features = np.array(iris_df.iloc[:, :-1])
labels = np.array(iris_df.iloc[:, -1])

# Encoder iris labels
label_names = np.unique(labels)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label_names)
label_dict = {}
for label, encoded_label in zip(label_names, encoded_labels):
   label_dict[label] = encoded_label

# Map Encoded labels to DF
labels = label_encoder.transform(labels)
print('Features shape:', features.shape, 'Labels shape:', labels.shape)

Features shape: (150, 4) Labels shape: (150,)


In [4]:
# Split the dataset into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size = 0.2, random_state = 42
)

##### Dataloader

In [5]:
batch_size = 16
train_dataset = IrisDataset(train_features, train_labels)
test_dataset = IrisDataset(test_features, test_labels)
# Create a DataLoader
train_dataloader = DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True, num_workers = 4
)
test_dataloader = DataLoader(
    test_dataset, batch_size = batch_size, shuffle = True, num_workers = 4
)

In [6]:
model = IrisCNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

##### Trainning

In [7]:
num_epochs = 100
for epoch in range(num_epochs):
    for features, labels in train_dataloader:
        optimizer.zero_grad()
        # Forward pass
        outputs = model(features)
        loss = loss_fn(outputs, labels)
        # Backward and optimize
        loss.backward()
        optimizer.step()
    
    # Print the training loss after each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/100], Loss: 1.0125
Epoch [2/100], Loss: 1.0338
Epoch [3/100], Loss: 0.8717
Epoch [4/100], Loss: 0.7971
Epoch [5/100], Loss: 0.7008
Epoch [6/100], Loss: 0.6612
Epoch [7/100], Loss: 0.6594
Epoch [8/100], Loss: 0.3917
Epoch [9/100], Loss: 0.1777
Epoch [10/100], Loss: 0.2509
Epoch [11/100], Loss: 0.3133
Epoch [12/100], Loss: 0.2605
Epoch [13/100], Loss: 0.2125
Epoch [14/100], Loss: 0.1513
Epoch [15/100], Loss: 0.4820
Epoch [16/100], Loss: 0.2557
Epoch [17/100], Loss: 0.1710
Epoch [18/100], Loss: 0.0850
Epoch [19/100], Loss: 0.0728
Epoch [20/100], Loss: 0.1118
Epoch [21/100], Loss: 0.0462
Epoch [22/100], Loss: 0.1054
Epoch [23/100], Loss: 0.0224
Epoch [24/100], Loss: 0.0167
Epoch [25/100], Loss: 0.0382
Epoch [26/100], Loss: 0.0945
Epoch [27/100], Loss: 0.0496
Epoch [28/100], Loss: 0.0110
Epoch [29/100], Loss: 0.2106
Epoch [30/100], Loss: 0.2897
Epoch [31/100], Loss: 0.0260
Epoch [32/100], Loss: 0.0518
Epoch [33/100], Loss: 0.1261
Epoch [34/100], Loss: 0.0239
Epoch [35/100], Loss: 0

##### Testing

In [8]:
with torch.no_grad():
    correct = 0
    total = 0
    for features, labels in test_dataloader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 96.66666666666667%
